<a href="https://colab.research.google.com/github/BrettonSteiner/cse380-notebooks/blob/master/ponder_and_prove_combinatorics_and_probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ponder and Prove Combinatorics and Probability
#### Due: Saturday, 6 February 2021, 11:59 pm.

## Conjecture

A number-theoretic conjecture of combinatorial significance is the following:

$degree2({2n \choose n}) =$ the "bits-on count" (or population count, or Hamming weight) of $n$.

$degree2(m)$ is defined as the number (degree, exponent) of 2's in the prime factorization of $m$.

In other words, for any $m$, a positive integer, $m = 2^e \cdot o$ where $o$ is an odd positive integer (could be 1) and $e$ is a natural number, including zero --- which would be the case when $m$ is odd. It's the $e$ that is the $degree2$ of $m$.

Another way to state this conjecture is that the number of 1's in the binary expansion of ${2n \choose n}$ for positive integer $n$ is equal to the number of 2's in the prime factorization of $n$.

Your task is to write Python code to test this conjecture for as many positive integers as you can. See the self-assessment for more details.

Note: a `bitsoncount` function can be a one-liner in Python: `return bin(x).count('1')`



### My ponder5.pyw file:

This file utilizes the multiprocessing library to speed up the program, a logger to take note of its progress every 30 minutes or so, and an internal timer that will automatically wrap-up the program after 24 hours have passed. I could probably make a better algorithm given more time, but this will have to do. It relies on Brother Neff's nCk.py file and Brother Comeau's cse251.py file to work. I've included them below.

In [ ]:
import time
import timeit
from itertools import repeat
import multiprocessing as mp
from nCk import *
from cse251 import *

# Leave 2 cpus free to keep the machine running :)
CPU_COUNT = mp.cpu_count() - 2

# Thanks Brother Neff!
def bits_on_count(x):
  return bin(x).count('1')

# This is a function derived from an example I found here:
# https://www.geeksforgeeks.org/count-occurrences-of-a-prime-number-in-the-prime-factorization-of-every-element-from-the-given-range/
def get_num_twos(n):
  prime = 2 # The prime we are looking for
  count = 0 
  val = prime
  while (True): 
    # Number of values in n that are divisible by val 
    a = n // val

    # Number of values in n - 1 that are divisible by val 
    b = (n - 1) // val

    # Increment the power of the val 
    val *= prime

    # (a - b) is the count of numbers in n that are divisible by val 
    if (a - b): 
      count += (a - b)

    # No values that are divisible by val 
    # thus exiting from the loop 
    else: 
      break

  return int(count); 

def test_conjecture(n):
  return bits_on_count(n) == get_num_twos(nCk(2*n,n))

def process_test_conjecture(n, eval_count, test_result, lock):
  if (test_result.value):
    result = test_conjecture(n)
    if (result):
      lock.acquire()
      eval_count.value += 1
      lock.release()
    else:
      lock.acquire()
      test_result.value = result
      lock.release()


if __name__ == '__main__':
  all_process_time = timeit.default_timer()
  log = Log(show_terminal=True)
  log.write(f"Began processing with {CPU_COUNT} processes")

  # 86400 seconds = 24 hours
  max_time = 86400
  start_time = time.time()
  n = 1
  display_timer = 1800
  display_count = 0
  manager = mp.Manager()
  test_result = manager.Value('test_result', True)
  eval_count = manager.Value('eval_count', 0)
  lock = manager.Lock()
  try:
    with mp.Pool(CPU_COUNT) as p:
      while (time.time() - start_time) < max_time and test_result:
        eval_count.value = 0
        amount = CPU_COUNT * (250 - int(n * 0.005))
        if (amount < CPU_COUNT):
          amount = CPU_COUNT
        numbers = range(n, n + amount)

        p.starmap(process_test_conjecture, zip(numbers, repeat(eval_count), repeat(test_result), repeat(lock)))

        n += eval_count.value

        if not test_result.value:
          break

        if (time.time() - start_time) < max_time and (time.time() - start_time) // display_timer > display_count:
          log.write(f'Verified up to {n}')
          display_count += 1

    if (time.time() - start_time) >= max_time:
      log.write('Time\'s up!')
    else:
      log.write('Conjecture evaluated to False!')
    log.write(f'Verified up to {n}')

    total_time = timeit.default_timer() - all_process_time
    hours = total_time  // 3600
    minutes = (total_time - (hours * 3600)) // 60
    seconds = total_time - (hours * 3600) - (minutes * 60)
    log.write(f'Total Time for ALL processing: {int(hours)} hours, {int(minutes)} minutes, and {seconds:.4f} seconds.')
  except:
    log.write(f'Verified up to {n}')

    total_time = timeit.default_timer() - all_process_time
    hours = total_time  // 3600
    minutes = (total_time - (hours * 3600)) // 60
    seconds = total_time - (hours * 3600) - (minutes * 60)
    log.write(f'Total Time for ALL processing: {int(hours)} hours, {int(minutes)} minutes, and {seconds:.4f} seconds.')

### My cse251.py file:

This file contains a subset of the classes in the original file. Thank you, Brother Comeau!

In [ ]:
"""
Course: CSE 251
File: cse251.py
Author: Brother Comeau

Purpose: Common classes for the cse 251 course

"""

import time
import logging
from datetime import datetime


# ===============================================================================================
class Log():
    """ Logger Class for CSE 251 """

    def __init__(self, filename_log='',
                 linefmt='',
                 show_levels=False,
                 show_terminal=False,
                 include_time=True):
        self._start_time = time.perf_counter()
        self._show_terminal = show_terminal

        if filename_log == '':
            d = datetime.now()
            localtime = d.strftime("%m%d-%H%M%S")
            filename_log = f'{localtime}.log'

        self._filename = filename_log

        if linefmt == '':
          linefmt = '%(message)s'

        if show_levels:
            linefmt = '%(levelname)s - ' + linefmt

        if include_time:
            date_format = '%H:%M:%S'
            linefmt = '%(asctime)s| ' + linefmt
        else:
            date_format = ''

        # Create and configure logger
        logging.basicConfig(filename=self._filename,
                            # format='%(asctime)s %(levelname)s %(message)s',
                            format=linefmt,
                            datefmt=date_format,
                            filemode='w')

        self.logger = logging.getLogger()

        # Setting the threshold of logger to DEBUG
        self.logger.setLevel(logging.INFO)

        if show_terminal:
            formatter = logging.Formatter(linefmt, datefmt=date_format)
            terminal_handler = logging.StreamHandler()
            terminal_handler.setFormatter(formatter)
            self.logger.addHandler(terminal_handler)


    def start_timer(self, message=''):
        """Start a new timer"""
        if message != '':
            self.write(message)
            
        self._start_time = time.perf_counter()

    def step_timer(self, message=''):
        """Current timer value"""
        t = time.perf_counter() - self._start_time
        if message == '':
            self.write(f'{t:0.8f}')
        else:
            self.write(f'{message} = {t:0.8f}')
        return t

    def stop_timer(self, message=''):
        """Stop the timer, and report the elapsed time"""
        t = time.perf_counter() - self._start_time
        if message == '':
            self.write(f'{t:0.8f}')
        else:
            self.write(f'{message} = {t:0.8f}')
        return t

    def write(self, message=''):
        """Write info message to log file"""
        self.logger.info(message)
        # if self._show_terminal:
        #   print(f'LOG: {message}')

    def write_warning(self, message=''):
        """Write warning message to log file"""
        self.logger.warning(message)
        # if self._show_terminal:
        #   print(f'LOG: {message}')

    def write_error(self, message=''):
        """Write error message to log file"""
        self.logger.error(message)
        # if self._show_terminal:
        #   print(f'LOG: {message}')


### My nCk.py file:

Thank you, Brother Neff!

In [ ]:
from math import gcd

def nCk(n, k):
  if k < 0 or k > n:
    return 0
  else:
    result = 1
    d = 1
    g = 1
    m = min(k, n - k)
    while (d <= m):
      g = gcd(result, d)
      result = n * (result // g)
      result = (result // (d // g))
      n -= 1
      d += 1
    return result

### Results

I ran my code for 24 hours on my desktop computer. Shown below are the log results. I had it report on its progress about every 30 minutes or so in the case that it failed to complete the 24-hour time frame.

```
11:42:24| Began processing with 14 processes
12:12:56| Verified up to 32663
12:43:09| Verified up to 41091
13:12:51| Verified up to 46859
13:42:28| Verified up to 51479
14:12:24| Verified up to 55413
14:42:25| Verified up to 58871
15:12:25| Verified up to 61965
15:42:24| Verified up to 64779
16:12:24| Verified up to 67369
16:42:24| Verified up to 69777
17:12:32| Verified up to 72045
17:42:35| Verified up to 74173
18:12:24| Verified up to 76175
18:42:27| Verified up to 78093
19:12:37| Verified up to 79927
19:42:24| Verified up to 81663
20:12:30| Verified up to 83343
20:42:26| Verified up to 84953
21:12:26| Verified up to 86507
21:42:24| Verified up to 88005
22:12:30| Verified up to 89461
22:42:40| Verified up to 90875
23:12:31| Verified up to 92233
23:42:36| Verified up to 93563
00:12:36| Verified up to 94851
00:42:43| Verified up to 96111
01:12:34| Verified up to 97329
01:42:29| Verified up to 98519
02:12:43| Verified up to 99695
02:42:33| Verified up to 100829
03:12:40| Verified up to 101949
03:42:42| Verified up to 103041
04:12:31| Verified up to 104105
04:42:36| Verified up to 105155
05:12:26| Verified up to 106177
05:42:28| Verified up to 107185
06:12:34| Verified up to 108179
06:42:50| Verified up to 109159
07:12:41| Verified up to 110111
07:42:39| Verified up to 111049
08:12:39| Verified up to 111973
08:42:42| Verified up to 112883
09:12:47| Verified up to 113779
09:42:51| Verified up to 114661
10:12:27| Verified up to 115515
10:42:29| Verified up to 116369
11:12:26| Verified up to 117209
11:42:48| Time's up!
11:42:48| Verified up to 118049
11:42:48| Total Time for ALL processing: 24 hours, 0 minutes, and 24.3162 seconds.
```

With my code running on my best machine, I was able to verify up to number 118049.

## Basic Probability Theory Question
A dark room contains two barrels. The first barrel is filled with green marbles, the second is filled with a half-and-half mixture of green and blue marbles. So there's a 100% chance of choosing a green marble from the first barrel, and a 50% chance of choosing either color in the second barrel. You reach into one of the barrels (it's dark so you don't know which one) and select a marble at random. It's green. You select another. It's green too. You select a third, a fourth, a fifth, etc. Green each time. What is the *minimum* number of marbles you need to select to *exceed* a probability of 99% that you are picking them out of the all-green barrel? (Note that there are enough marbles so that the answer does not depend on how many marbles are in the second barrel.)


The following equations came from the Hint section on page 175 of the *First Three Odds* textbook.

The probability of choosing a **green** marble from the
- all-green barrel $= 1$
- half-and-half barrel $= (1/2)^n$

The probability of choosing a **blue** marble from the
- all-green barrel $= 0$
- half-and-half barrel $= 1-(1/2)^n$

I can use the equation for the probability of getting a blue marble from the half-and-half barrel to tell at what number of marbles I choose $n$ at which I will have more than a 99% probability of getting a blue marble.

In [4]:
def green_marble_probability(n):
  return (1/2) ** n

def blue_marble_probability(n):
  return 1 - (1/2) ** n

for n in range(10):
  print(f'n: {n}')
  print(f'green: {green_marble_probability(n)}')
  print(f'blue: {blue_marble_probability(n)}')
  print()

n: 0
green: 1.0
blue: 0.0

n: 1
green: 0.5
blue: 0.5

n: 2
green: 0.25
blue: 0.75

n: 3
green: 0.125
blue: 0.875

n: 4
green: 0.0625
blue: 0.9375

n: 5
green: 0.03125
blue: 0.96875

n: 6
green: 0.015625
blue: 0.984375

n: 7
green: 0.0078125
blue: 0.9921875

n: 8
green: 0.00390625
blue: 0.99609375

n: 9
green: 0.001953125
blue: 0.998046875



I will have a probability greater than 99% that I am pulling marbles from the all-green barrel if I consecutively choose seven or more green marbles from the barrel.

## A Related But Deeper Basic Probability Theory Question
Take a deep breath. Suppose Shakespeare's account is accurate and Julius Caesar gasped "You too, Brutus" before breathing his last. What is the probability that you just inhaled a molecule that Julius Caesar exhaled in his dying breath?

Assume that after more than two thousand years the exhaled molecules are uniformly spread about the world and the vast majority are still free in the atmosphere. Assume further that there are $10^{44}$ molecules of air in the world, and that your inhaled quantity and Caesar's exhaled quantity were each about $2.2 \times 10^{22}$ molecules.
### Hint
If a number $x$ is small, then $(1 - x)$ is approximately equal to $e^{-x}$.


In [56]:
from math import e

# The amount of molecules on a single breath
breath_quantity = (2.2 * (10 ** 22))

print('The probability of a single molecule being from Julius Caesar\'s dying breath:')
pr_caesars_breath = breath_quantity / (10 ** 44)
print(pr_caesars_breath)
print()

print('The probability that no molecules in my breath are from Julius Caesar\'s dying breath:')
pr_none = e ** (-pr_caesars_breath * breath_quantity)
print(pr_none)
print()

print('The probability that at least one molecule in my breath is from Julius Caesar\'s dying breath:')
print(1 - pr_none)
print()

The probability of a single molecule being from Julius Caesar's dying breath:
2.1999999999999996e-22

The probability that no molecules in my breath are from Julius Caesar's dying breath:
0.00790705405159345

The probability that at least one molecule in my breath is from Julius Caesar's dying breath:
0.9920929459484066



The probability of inhaling a molecule of Julius Caeser's exhale from his dying breath is about 99.2%, which is far more likely than I was expecting!

Daniel Strickland, Claire Hocker, and I used this reference to help us figure it out: https://puzzlemath.blogspot.com/2011/06/julius-caesars-last-breath.html

## What is True?
Assess yourself on how you did using the checkboxes below. Check a box by putting an 'X' in it only if it is warranted.


### What is true of my experience in general?
(5 points each, 15 points total)
- [X] I had fun.
- [X] I learned something new.
- [X] I achieved something meaningful, or something I can build upon at a later time.

I can say I did all of these when I implemented my program using multiprocessing and logging. I am currently taking a parallelism and concurrency class, so I will likely use this going forward. I even learned about the starmap function which I have never used before. It was a lot of fun, but I probably spent too much time on it.

### What is true of my report on what I learned?
(5 points each, 25 points total)
- [X] I wrote a sufficient number of well-written sentences.
- [X] My report is free of "mechanical infelicities" (misspelled words, grammatical errors, punctuation errors, etc.).
- [X] I reported on any connections I found between this investigation and something I already know.
- [X] I reported who were and what contribution each of my collaborators made.
- [X] I reported on how many numbers I was able to verify with a time/computation budget of 24 hours (in a row).

I believe I have accomplished all of these because I have nothing more to write and everything that I have written has been run through Grammarly. I also mentioned above about my class on parallelism and concurrency that I have utilized in this class.

### What is true about my answers?
(15 points each, 60 points total)
- [X] I figured out how to run a Python program continuously for at least 24 hours.
- [ ] I refrained from printing out anything except the highest number I verified, knowing that printing just slows a program down.
- [X] I got the right answer for the first probability theory question.
- [X] I got the right answer for the second probability theory question.

I believe I achieved all of these except for the printing out just one number part. This is proof that I should be reading these requirements ahead of time. I could have done this but was unaware that it was a requirement. I actually went out of my way to print out the number at a set time about every 30 minutes as insurance against a program crash, so if anything, I think I went above and beyond this requirement. But for the sake of following the requirements to the letter, I will leave it unchecked. Feel free to give me credit for it if you see fit.

I worked with my classmates to work out the two probability theory questions and we are quite certain that we got the correct answer. We know for sure that we got the first one right because we got the same answer as was listed in the *First Three Odds* on page 175. The second was more challenging, but with the help of some research, we believe this one is correct, too.